추출한 엔트로피를 학습이 가능하게 CSV 파일로 변환하는 Notebook

입력파일
* NTInfo DiE로 추출한 엔트로피 (json)
* 악성여부가 표기된 CSV 파일

출력파일
* 패킹 여부와 엔트로피, 악성여부가 표기된 CSV 파일

In [10]:
import json
import csv
import glob
from tqdm.auto import tqdm
import parmap
import os
import re

In [11]:
# 추출한 엔트로피 파일 경로
entropy_path = input()

if entropy_path == '' or os.path.isdir(entropy_path) == False:
    print('Please input path of entropy files')
    raise FileNotFoundError(f'{entropy_path} is not found')

# 추출할 파일 경로 내의 PE 파일 목록
files = glob.glob(entropy_path + '/*.json')

In [14]:
csv_path = input()

if csv_path == '' or os.path.isfile(csv_path) == False:
    print('Please input path of CSV file for labeling')
    raise FileNotFoundError(f'{csv_path} is not exist')

# CSV 파일을 읽은 후 dict 형태로 저장 key는 sha256, value는 라벨
global label_dict
with open(csv_path, 'r') as f:
    reader = csv.reader(f)
    label_dict = {rows[0]: rows[1] for rows in reader}

In [15]:
def get_row(file):
    global label_dict
    basename_without_ext = os.path.basename(file).split('.')[0]
    with open(file, 'r') as f:
        data = json.load(f)
    is_malware = label_dict[basename_without_ext]

    return (1 if data['status'] == 'packed' else 0, data['total'], int(is_malware))

In [16]:
column_name = ['packed', 'entropy', 'malware']

result = parmap.map(get_row, files, pm_pbar=True, pm_chunksize=1)

  0%|          | 0/50000 [00:00<?, ?it/s]

In [17]:
column = []
column.append(column_name)
for r in result:
    column.append(list(r))

In [18]:
# save csv
with open('entropy.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(column)